In [ ]:
#########
#
# Notebook to turn randomized SYNEX data (after tiling) into a base data frame.
# This is a much easier data structure to manipulate and takes much less time to load
# and analyse than loading all the detector savefiles after gwemopt is run.
# This also adds a layer of security. If you accidentally overwrite a dataframe or
# permanently change it, then you till still have the base dat and lisabeta files to
# recreate teh dataframe from.
#
#########

In [ ]:
%matplotlib inline

# Set path to SYNEX
import os,sys
import pathlib
SYNEX_PATH=os.popen('pwd').read().split("SYNEX")[0]+"SYNEX"
sys.path.insert(1, SYNEX_PATH)
# print(sys.path)
# sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [ ]:
import importlib
import numpy as np
import pandas as pd
import lisabeta.utils.plotutils as plotutils
import time
import glob
import copy
from astropy.cosmology import Planck13, z_at_value
from astropy.cosmology import WMAP9 as cosmo
from astropy.time import Time
import json
import healpy as hp
import gwemopt
from gwemopt import utils as gou
import SYNEX
from SYNEX import SYNEX_Utils as SYU
from SYNEX import SYNEX_Detectors as SYDs
from SYNEX import SYNEX_Sources as SYSs
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pylab as pylab
from SYNEX.SYNEX_Utils import pylab_params
pylab.rcParams.update(pylab_params)
importlib.reload(SYU)

In [ ]:
# Grab all available systems
AthenaSaves=glob.glob(SYNEX_PATH+"/Saved_Telescope_Dicts/Randomized_SYNEX2/Randomized_*.dat")
print(len(AthenaSaves),"randomized SYNEX Athena savefiles found. Forming list of telescope objects [this may take some time]...")
Athenas=[SYDs.Athena(**{"ExistentialFileName":f,"verbose":False}) for f in AthenaSaves]

# Remove faulty stuff that needs replacing
Athenas=[a for a in Athenas if a.detector_source_coverage["source JsonFile"]!=None]

print("DONE.")


In [ ]:
# Check savefile substructures (these might be different on cluster and therefore need changing to local copies of objects.)
print(AthenaSaves[0],Athenas[0].ExistentialFileName,Athenas[0].detector_source_coverage["source save file"],Athenas[0].detector_source_coverage["source JsonFile"],Athenas[0].detector_source_coverage["source H5File"])
print("\n")

# Need to change location of Json files since paths are simple on cluster but have substucture locally
for ii in range(len(Athenas)):
    Athenas[ii].detector_source_coverage["source JsonFile"] = SYNEX_PATH + "/inference_param_files/Randomized_SYNEX2/" + Athenas[ii].detector_source_coverage["source JsonFile"].split("/")[-1]
    Athenas[ii].detector_source_coverage["source H5File"] = SYNEX_PATH + "/inference_data/Randomized_SYNEX2/" + Athenas[ii].detector_source_coverage["source H5File"].split("/")[-1]

# Check if things are good now
print(AthenaSaves[0],Athenas[0].ExistentialFileName,Athenas[0].detector_source_coverage["source save file"],Athenas[0].detector_source_coverage["source JsonFile"],Athenas[0].detector_source_coverage["source H5File"])


In [ ]:
# Create data frame from Athena objects
SaveFile = 'Randomized_SYNEX2/data.h5'
data = SYU.GetDataFrame(detectors=Athenas,SaveFile=SaveFile) # Both inputs given so any existing SaveFile will be overwritten. 
print(data.head(10))
